<a href="https://colab.research.google.com/github/caro28/stinky/blob/master/oil_vessel_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

*   This file loads and merges two xlsx spreadsheets, which include oil vessel arrival data into Portland for 2020 and 2021.
* Running all cells (Runtime --> Run all) will automatically download df_vessels.csv, on the user's computer, after loading and merging the raw data.
*   New columns were added to df_vessels.csv to look at dates more granularly.

# Instructions for updating:
First, in Excel: user uploading data from Excel should do the following first:

1. data should be on one sheet only. if data is pulled from other sheets in Excel file, user should copy/paste (values only) data in place, then delete all other sheets.
2. delete empty rows above column names.
3. column names should be in one row only. Unmerge any merged rows that contain column names. Delete all unmerged rows that do not contain the main column name.
4. ensure column names match the order of the sequence below. If not, then move columns so that they remain in this order. Do not rename columns to match names below; this notebook will rename them.

> 'ARRIVAL DATE', 'ARRIVAL TIME (if avail)', 'TERMINAL', 'SHIP NAME', 'LAST PORT OF CALL (if avail)', 'PRODUCT TYPE - OIL', 'PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)', 'PRODUCT AMOUNT - OIL, Outgoing', 'RUNNING TOTAL (Oil)', 'PRODUCT TYPE - NON-OIL',	'PRODUCT AMOUNT - NON OIL, Incoming', 'PRODUCT AMOUNT - NON OIL, Outgoing', 'Running Total (Specify units) (Non-Oil)'

5. check that all dates have been entered correctly (e.g. original 2021 Excel file contained date "2/13/20221" - errors such as this should be fixed in Excel to 2/13/2021).
6. Delete any empty rows at the end of the sheet.
7.   save as csv.

Second, on GitHub:

1. If receive new data in a new spreadsheet: first, go through steps above in Excel.
2. Upload new spreadsheet (csv) to [vessels_data_raw folder](https://github.com/ds5110/stinky/tree/master/vessels_data). Filename does not matter but must end in '.csv'.
3. Open this notebook in Google colab.
4. Follow instructions in cell 1 ("How to add a new csv file to code").
4. Runtime --> Run all: this will download the updated df_vessels table.
5. Save as df_vessels.csv and upload to [data folder](https://github.com/ds5110/stinky/tree/master/data). (Do not need to delete old df_vessels.csv file; GitHub will automatically replace with new file.)


# How to add a new csv file to code
* In cell immediately below, make two changes:
1. add new url, using this format: url_name = 'URL'
  * to get 'URL', go to [vessels_data_raw folder](https://github.com/ds5110/stinky/tree/master/vessels_data) and click on new file.
  * click on "Raw" (top right side of page)
  * copy URL and paste in cell below

2. add url_name to url_lst

Examples of both changes are provided in comments in cell below.

In [27]:
import pandas as pd

# new csv files with raw data: add url below (see example below)
url_2020 = 'https://raw.githubusercontent.com/ds5110/stinky/master/vessels_data_raw/2020%20SMRO%20VESSEL%20ARRIVALS.csv'
url_2021 = 'https://raw.githubusercontent.com/ds5110/stinky/master/vessels_data_raw/2021%20SMRO%20VESSEL%20ARRIVALS.csv'
# url_2022 = 'https://raw.githubusercontent.com/ds5110/stinky/master/vessels_data_raw/{NEW_FILENAME}.csv' # example of how to add url to new raw data file:

# new csv files with raw data: add url name to list below (see example below)
url_lst = [url_2020, url_2021]
# url_lst = [url_2020, url_2021, url_2022] # example of how to update url_lst after adding url_2022:

def get_data(urls):
  '''
  Function: get_data()
    Retrieves url's to csv files from a list, loads them with pandas into 
    DataFrames, concatenates them, then returns a single, merged DataFrame.
  Parameters:
    url_lst: (list) list of url's to csv data files
  Returns:
    (pd.DataFrame) data from csv files merged into single DataFrame
  '''
  collector = []
  for url in urls:
    collector.append(pd.read_csv(url, header=None, skiprows=1))
  return pd.concat(collector, ignore_index=True)

df_vessels = get_data(url_lst)
print(df_vessels.shape)

(770, 13)


In [28]:
# rename columns
col_names = ['ARRIVAL DATE', 'ARRIVAL TIME (if avail)', 'TERMINAL', 
             'SHIP NAME', 'LAST PORT OF CALL (if avail)', 'PRODUCT TYPE - OIL',
             'PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)',
             'PRODUCT AMOUNT - OIL, Outgoing', 'RUNNING TOTAL (Oil)',
             'PRODUCT TYPE - NON-OIL', 'PRODUCT AMOUNT - NON OIL, Incoming',
             'PRODUCT AMOUNT - NON OIL, Outgoing', 'Running Total (Specify units) (Non-Oil)']

df_vessels.columns = col_names

In [38]:
import string

def separate_date_cols(df):
  '''
  Function: separate_date_cols()
    Converts 'Arrival Date' column to datetime object and creates new, separate 
    columns for year, month, day, and month_name
  Parameters:
    df: (pandas.DataFrame object) input dataframe (df_vessels)
  Returns:
    None
  '''
  df['ARRIVAL DATE']=pd.to_datetime(df['ARRIVAL DATE'])
  df['Year']=df['ARRIVAL DATE'].dt.year
  df['Month']=df['ARRIVAL DATE'].dt.month
  df['Day']=df['ARRIVAL DATE'].dt.day
  df['Month_name'] = pd.to_datetime(df['Month'], format='%m').dt.month_name().str.slice(stop=3)

def strip_char(df):
  '''
  Function: strip_char(df)
    Removes digits and special characters from 'Product type - oil' column
  Parameters:
    df: (pandas.DataFrame object) input dataframe (df_vessels)
  Returns:
    None
  '''
  #df['PRODUCT TYPE - OIL'] = df['PRODUCT TYPE - OIL'].str.lstrip(string.digits) # Aishwarya will edit this line to keep product codes but edit formatting of 01, 02, 06 to match all other codes
  df['PRODUCT TYPE - OIL'] = df['PRODUCT TYPE - OIL'].replace(['#'],'',regex=True)

In [30]:
# create separate columns for year/month/day
separate_date_cols(df_vessels)

# strip characters from 'Product Type - Oil"
strip_char(df_vessels)

In [31]:
# sort chronologically
df_vessels=df_vessels.sort_values(by='ARRIVAL DATE')

# reset index
df_vessels=df_vessels.reset_index(drop=True)

In [32]:
def convert_float(col):
  '''
  Function: convert_float()
  Parameters:
    col: (pandas.Series object) column whose values will be converted to float
  Returns:
    (list) column values in a list, converted to float
  '''
  return [float(str(x).replace(',', '')) for x in col]

def replace_nan_0(col):
  '''
  Function: replace_nan()
    Replaces any NaN values in input with 0
  Parameters:
    col: (pandas.Series object) column whose NaN values will be replaced with 0
  Returns:
    None
  '''
  col.fillna(0, inplace=True)

In [33]:
# convert numerical data columns to float
df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)'] = convert_float(df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)'])
df_vessels['PRODUCT AMOUNT - OIL, Outgoing'] = convert_float(df_vessels['PRODUCT AMOUNT - OIL, Outgoing'])
df_vessels['Running Total (Specify units) (Non-Oil)'] = convert_float(df_vessels['Running Total (Specify units) (Non-Oil)'])

# replace NaN with 0
replace_nan_0(df_vessels['PRODUCT AMOUNT - OIL, Outgoing'])
replace_nan_0(df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)'])
replace_nan_0(df_vessels['PRODUCT AMOUNT - NON OIL, Incoming'])
replace_nan_0(df_vessels['PRODUCT AMOUNT - NON OIL, Outgoing'])

In [34]:
def correct_runningtotal(incoming, outgoing):
  '''
  Function: correct_runningtotal()
    Re-calculates values in 'Running Total' columns (net cumulative sums), due 
    to errors in original values in Excel files and to maintain the cumulative
    sum across merged spreadsheets.
  Paramaters:
    incoming: (pandas.Series object) column with values to be added to 
      cumulative sum
    outgoing: (pandas.Series object) column with values to be subtracted from 
      cumulative sum
  Returns:
    running_total: (list) new values for 'Running total' columns
  '''
  running_total = []
  net = incoming[0]
  running_total.append(net)
  for i in range(1, len(df_vessels)):
    net += incoming[i]
    net -= outgoing[i]
    running_total.append(net)
  return running_total

In [35]:
# correct 'Running total (Oil)'
incoming = df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)']
outgoing = df_vessels['PRODUCT AMOUNT - OIL, Outgoing']
df_vessels['RUNNING TOTAL (Oil)'] = correct_runningtotal(incoming, outgoing)

# correct 'Running total (Non-oil)'
incoming = df_vessels['PRODUCT AMOUNT - NON OIL, Incoming']
outgoing = df_vessels['PRODUCT AMOUNT - NON OIL, Outgoing']
df_vessels['Running Total (Specify units) (Non-Oil)'] = correct_runningtotal(incoming, outgoing)

In [36]:
df_vessels.isna().sum()

ARRIVAL DATE                                                                      0
ARRIVAL TIME (if avail)                                                         770
TERMINAL                                                                          0
SHIP NAME                                                                         0
LAST PORT OF CALL (if avail)                                                     67
PRODUCT TYPE - OIL                                                                0
PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)      0
PRODUCT AMOUNT - OIL, Outgoing                                                    0
RUNNING TOTAL (Oil)                                                               0
PRODUCT TYPE - NON-OIL                                                          770
PRODUCT AMOUNT - NON OIL, Incoming                                                0
PRODUCT AMOUNT - NON OIL, Outgoing                                          

In [37]:
# download tidied df to user's computer (local files)
from google.colab import files
df_vessels.to_csv('df_vessels.csv', index=False)
files.download('df_vessels.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>